# Otimização Pandas

In [3]:
# importando libs necessarias
import pandas as pd
import numpy as np

In [87]:
# Funcao que mede o uso de memoria de cada coluna e do df
def memory(df):
    memStats = df.memory_usage()
    print("Consumo de cada coluna em bytes:")
    print(memStats)
    print("Consumo do DataFrame em bytes:%d bytes"%(memStats.sum()))
    print("Consumo do DataFrame em megabytes(MB): %2.2f MB"%(memStats/1024 ** 2).sum())
    return memStats.sum()

In [119]:
# importando aquivo csv com o pandas
df = pd.read_csv('SP_poluicao_dados.csv')

In [120]:
# excluindo colunas duplicatadas
df.drop(['Unnamed: 0','ID'], axis=1, inplace=True)

In [121]:
# verificando as 5 primeiras linhas do DataFrame
df.head()

,Data,Hora,Estacao,Codigo,Poluente,Valor,Unidade,Tipo
0,2015-01-01,01:00,Americana - Vila Santa Maria,SP01,MP10,65.0,ug/m3,automatica
1,2015-01-01,02:00,Americana - Vila Santa Maria,SP01,MP10,98.0,ug/m3,automatica
2,2015-01-01,03:00,Americana - Vila Santa Maria,SP01,MP10,79.0,ug/m3,automatica
3,2015-01-01,04:00,Americana - Vila Santa Maria,SP01,MP10,53.0,ug/m3,automatica
4,2015-01-01,05:00,Americana - Vila Santa Maria,SP01,MP10,35.0,ug/m3,automatica


In [122]:
m1 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        87844120
Hora        87844120
Estacao     87844120
Codigo      87844120
Poluente    87844120
Valor       87844120
Unidade     87844120
Tipo        87844120
dtype: int64
Consumo do DataFrame em bytes:702753088 bytes
Consumo do DataFrame em megabytes(MB): 670.20 MB


## Regras de Otimização
https://wiki.c2.com/?RulesOfOptimizationClub
 1. You do not optimize.
 2. You do not optimize, without measuring first.
 3. When the performance is not bound by the code, but by external factors, the optimization is over.
 4. Only optimize code that already has full unit test coverage.
 5. One factor at a time.
 6. No unresolved bugs, no schedule pressure.
 7. Testing will go on as long as it has to.
 8. If this is your first night at Optimization Club, you have to write a test case.

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980515 entries, 0 to 10980514
Data columns (total 8 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Data      object 
 1   Hora      object 
 2   Estacao   object 
 3   Codigo    object 
 4   Poluente  object 
 5   Valor     float64
 6   Unidade   object 
 7   Tipo      object 
dtypes: float64(1), object(7)
memory usage: 670.2+ MB


In [125]:
df['Estacao'] = df['Estacao'].astype('category')

In [126]:
m2 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        87844120
Hora        87844120
Estacao     10983315
Codigo      87844120
Poluente    87844120
Valor       87844120
Unidade     87844120
Tipo        87844120
dtype: int64
Consumo do DataFrame em bytes:625892283 bytes
Consumo do DataFrame em megabytes(MB): 596.90 MB


In [127]:
print(f'{(m1 - m2) / (1024 ** 2):.2f} MB')
print(f'{(m1-m2)/m1*100:.2f}%')

73.30 MB
10.94%


In [128]:
df['Codigo'].nunique()

87

In [129]:
df['Codigo'] = df['Codigo'].astype('category')
m3 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        87844120
Hora        87844120
Estacao     10983315
Codigo      10983315
Poluente    87844120
Valor       87844120
Unidade     87844120
Tipo        87844120
dtype: int64
Consumo do DataFrame em bytes:549031478 bytes
Consumo do DataFrame em megabytes(MB): 523.60 MB


In [130]:
for coluna in ['Poluente','Unidade','Tipo']:
    print(coluna)
    print(f'Numero de unicos {df[coluna].nunique()}, {(df[coluna].nunique() / len(df)) * 100 :.7f}%')
    print('------------')

Poluente
Numero de unicos 9, 0.0000820%
------------
Unidade
Numero de unicos 1, 0.0000091%
------------
Tipo
Numero de unicos 2, 0.0000182%
------------


In [133]:
for coluna in ['Poluente','Unidade','Tipo']:
    df[coluna] = df[coluna].astype('category')

m4 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        87844120
Hora        87844120
Estacao     10983315
Codigo      10983315
Poluente    10980887
Valor       87844120
Unidade     10980631
Tipo        10980597
dtype: int64
Consumo do DataFrame em bytes:318441233 bytes
Consumo do DataFrame em megabytes(MB): 303.69 MB


In [163]:
df['Valor'] = df['Valor'].astype('float32')
m5 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        87844120
Hora        87844120
Estacao     10983315
Codigo      10983315
Poluente    10980887
Valor       43922060
Unidade     10980631
Tipo        10980597
dtype: int64
Consumo do DataFrame em bytes:274519173 bytes
Consumo do DataFrame em megabytes(MB): 261.80 MB


In [166]:

df['Data'].describe()
df['Hora'].describe()

count     10980515
unique          24
top          17:00
freq        469259
Name: Hora, dtype: object

In [ ]:
df['Data'] = df['Data'].astype('category')
df['Hora'] = df['Hora'].astype('category')
m6 = memory(df)

Consumo de cada coluna em bytes:
Index            128
Data        22047574
Hora        10981263
Estacao     10983315
Codigo      10983315
Poluente    10980887
Valor       43922060
Unidade     10980631
Tipo        10980597
dtype: int64
Consumo do DataFrame em bytes:131859770 bytes
Consumo do DataFrame em megabytes(MB): 125.75 MB


In [176]:
print(f'Inicial: {m1 / (1024 ** 2):.2f} MB')
print(f'Final: {m6 / (1024 ** 2):.2f} MB')
print(f'Diferença: {(m1 - m6) / (1024 ** 2):.2f} MB')
print(f'Diminuição de {(m1-m6)/m1*100:.2f}%')
print(f'M6 enquivale a {m6/m1*100:.2f}% do M1')

Inicial: 670.20 MB
Final: 125.75 MB
Diferença: 544.45 MB
Diminuição de 81.24%
M6 enquivale a 18.76% do M1


# 5X MENOR